# Settings

In [4]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier

In [7]:
os.chdir('/content/gdrive/My Drive/피셋 방세/dataset')

# 모델 불러오기

In [9]:
import joblib

In [10]:
lg = joblib.load('final_model.pkl')

# 데이터 불러오기

In [17]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_feature.csv') #test셋 처음 등장!

In [43]:
# 미리 전처리한 train셋: test셋에 컬럼명을 붙이기 위해 불러놓음.
X_train = pd.read_csv('knn_nopca_x_smote_final2.csv')
X_train = X_train.drop(['Unnamed: 0'], axis=1)

# 최종 test 데이터 pred

## 1. Preprocessing - test data

### 1.1 column 지워주기

아까 train 데이터에서 지웠던 column들을 test에서도 지워줌

In [18]:
# Train 데이터에서 NA 비율이 50% 이상이었던 column날리기
train_x = train.drop(['class'], axis=1)

# train_x column들의 NA 비율 구하기
na_ratio_col = train_x.isnull().sum().sort_values(ascending=False)/len(train_x)

# 삭제할 column들의 이름 모음
col_del = [ na_ratio_col.index[i]  for i in range(0,len(na_ratio_col)) if ( na_ratio_col[i] > 0.5 ) == True]

In [32]:
col_del

['br_000',
 'bq_000',
 'bp_000',
 'cr_000',
 'ab_000',
 'bo_000',
 'bn_000',
 'bm_000']

In [19]:
test.drop(col_del, axis=1, inplace=True)

In [33]:
test

,aa_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,ag_004,ag_005,ag_006,ag_007,ag_008,ag_009,ah_000,ai_000,aj_000,ak_000,al_000,am_0,an_000,ao_000,ap_000,aq_000,ar_000,as_000,at_000,au_000,av_000,ax_000,ay_000,ay_001,ay_002,ay_003,ay_004,ay_005,ay_006,ay_007,ay_008,...,db_000,dc_000,dd_000,de_000,df_000,dg_000,dh_000,di_000,dj_000,dk_000,dl_000,dm_000,dn_000,do_000,dp_000,dq_000,dr_000,ds_000,dt_000,du_000,dv_000,dx_000,dy_000,dz_000,ea_000,eb_000,ec_00,ed_000,ee_000,ee_001,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,82,6.800000e+01,40.0,0.0,0.0,0.0,0.0,0.0,0.0,748.0,12594.0,3636.0,0.0,0.0,0.0,5244.0,0.0,60.0,0.0,0.0,0.0,23174.0,18166.0,23686.0,1270.0,0.0,0.0,0.0,0.0,12.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,692.0,16286.0,...,0.0,7918.0,78.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,352.0,0.0,0.0,3996.0,584.0,200.0,62.0,37580.0,3756.0,6368.0,36.0,0.0,0.0,0.0,2.86,102.0,10040.0,3310.0,1068.0,276.0,1620.0,116.0,86.0,462.0,0.0,0.0,0.0,0.0
1,66002,2.120000e+02,112.0,0.0,0.0,0.0,0.0,0.0,199486.0,1358536.0,1952422.0,452706.0,25130.0,520.0,0.0,1891670.0,0.0,0.0,0.0,77898.0,110548.0,3605894.0,3291610.0,959756.0,286536.0,0.0,0.0,0.0,0.0,106.0,340.0,0.0,0.0,0.0,0.0,0.0,24286.0,681260.0,2808472.0,474782.0,...,0.0,3613906.0,4218.0,692.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25278.0,9438.0,2504.0,10262714.0,1278664.0,109700.0,19072.0,9520.0,4902.0,4434614.0,70900.0,0.0,0.0,26002880.0,2057.84,2158.0,396312.0,538136.0,495076.0,380368.0,440134.0,269556.0,1315022.0,153680.0,516.0,0.0,0.0,0.0
2,174,2.600000e+01,24.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7396.0,0.0,134.0,0.0,0.0,0.0,22666.0,19998.0,14550.0,788.0,0.0,0.0,0.0,0.0,40.0,124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,12438.0,52.0,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,230.0,0.0,0.0,6716.0,2274.0,610.0,126.0,920.0,242.0,8862.0,20.0,0.0,0.0,0.0,131.70,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,120278,2.130706e+09,256.0,0.0,0.0,0.0,0.0,0.0,0.0,208.0,61582.0,3858106.0,3553512.0,174722.0,13728.0,3242508.0,0.0,0.0,0.0,0.0,0.0,6749710.0,5825696.0,1123618.0,814174.0,0.0,0.0,0.0,0.0,374.0,90.0,0.0,0.0,0.0,0.0,0.0,3722.0,114864.0,4107068.0,3436204.0,...,22.0,6890130.0,4936.0,1114.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50196.0,49662.0,22016.0,0.0,0.0,226956.0,53298.0,22559560.0,2430612.0,0.0,0.0,0.0,0.0,80277450.0,1938.02,2248.0,836724.0,1682254.0,1006722.0,459658.0,876356.0,689532.0,568540.0,500624.0,1006628.0,34820.0,0.0,0.0
4,30758,4.500000e+02,402.0,0.0,0.0,0.0,0.0,0.0,0.0,44326.0,813956.0,979346.0,106420.0,12658.0,1130.0,1053414.0,0.0,108.0,0.0,0.0,0.0,1782786.0,1567350.0,516596.0,170742.0,0.0,0.0,0.0,0.0,1428.0,216.0,0.0,0.0,0.0,0.0,0.0,17514.0,1046172.0,597456.0,296694.0,...,0.0,1777396.0,1154.0,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15570.0,38472.0,5962.0,0.0,0.0,57216.0,12210.0,35960.0,8448.0,0.0,0.0,0.0,0.0,29759210.0,798.26,802.0,205190.0,419572.0,278338.0,123232.0,221032.0,184386.0,164702.0,133338.0,222050.0,5996.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,374,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,0.0,7122.0,18184.0,3002.0,0.0,0.0,0.0,12026.0,0.0,0.0,0.0,0.0,0.0,27648.0,24958.0,5828.0,1016.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24812.0,2186.0,1310.0,...,NaN,NaN,82.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,212.0,8.0,2.0,0.0,0.0,614.0,130.0,22840.0,2566.0,0.0,36.0,0.0,0.0,0.0,NaN,NaN,7636.0,4514.0,1104.0,268.0,556.0,13724.0,506.0,0.0,0.0,0.0,0.0,0.0
7996,40262,1.802000e+03,1720.0,0.0,0.0,0.0,0.0,0.0,0.0,10808.0,121360.0,2444244.0,880466.0,40288.0,0.0,1492318.0,0.0,0.0,0.0,0.0,0.0,3158490.0,2766430.0,474174.0,306270.0,0.0,0.0,0.0,0.0,1896.0,1398.0,0.0,0.0,0.0,0.0,0.0,2064.0,153262.0,1266388.0,2075452.0,...,46.0,2479178.0,4186.0,760.0,0.0,0.0,0

### 1.2 Scaling

train에 fit해줫던거 바탕으로 여기에서는 transform만

In [20]:
# train 에서 scaling 했던거 다시 구현
na_ratio_col = train_x.isnull().sum().sort_values(ascending=False)/len(train_x)
na_ratio_row = train_x.isnull().sum(1).sort_values(ascending=False)/len(train_x.columns)
col_del = [ na_ratio_col.index[i]  for i in range(0,len(na_ratio_col)) if ( na_ratio_col[i] > 0.5 ) == True]
train_x.drop(col_del, axis=1, inplace=True)
len(train_x.columns)
row_del = [ na_ratio_row.index[i] for i in range(0,len(na_ratio_row)) if (na_ratio_row[i] <= 0.5 ) ]
train_x = train_x.iloc[row_del,:]
len(train_x)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# train_x로 fit_transform 해주고
train_x_scaled = pd.DataFrame(scaler.fit_transform(train_x), columns = train_x.columns)

In [21]:
test_scaled = pd.DataFrame(scaler.transform(test),columns=test.columns)

### 1.3 KNN Impute

In [22]:
from sklearn.impute import KNNImputer

knn = KNNImputer(n_neighbors=5)

# Scaling 때와 마찬가지로 train을 fit_transform 해주고
train_x_knn = knn.fit_transform(train_x_scaled)

In [23]:
knn_X_test = knn.transform(test_scaled)
knn_X_test

array([[2.98554849e-05, 3.19142945e-08, 4.65967062e-09, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.40307526e-02, 9.94975064e-08, 1.30470777e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.33518826e-05, 1.22025244e-08, 2.79580237e-09, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [7.81339885e-04, 9.99999891e-01, 3.49475297e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.83597251e-02, 1.94301734e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.98015994e-02, 9.99999891e-01, 1.03910655e-07, ...,
        6.71009885e-03, 0.00000000e+00, 0.00000000e+00]])

여기에서도 transform만 해줌

In [44]:
knn_test_df = pd.DataFrame(knn_X_test, columns=X_train.columns)

## 2. Predict

In [49]:
pred_final = lg.predict(knn_test_df)

In [50]:
pred_final

array([0, 0, 0, ..., 0, 1, 0])

0과 1로 예측되었기 때문에 0.5 이상은 pos, 0.5 이하는 neg로 바꿔줌

In [51]:
pred_final = pd.DataFrame(pred_final)

In [52]:
pred_final ['class'] = [ 'pos' if pred_final[0][i]>0.5 else 'neg' for i in range(0,len(pred_final))]
pred_final

,0,class
0,0,neg
1,0,neg
2,0,neg
3,0,neg
4,0,neg
...,...,...
7995,0,neg
7996,0,neg
7997,0,neg
7998,1,pos


In [53]:
pred_final.drop([0], axis=1, inplace=True)

In [54]:
pred_final['id'] = list(range(1,8001))
pred_final.set_index(['id'])

,class
id,
1,neg
2,neg
3,neg
4,neg
5,neg
...,...
7996,neg
7997,neg
7998,neg


In [56]:
pred_final.to_csv('submission_final2.csv', index=False)